# Practico 2: Lorenz

**1)** Implemente una función que retorne el lado derecho de la ODE de Lorenz

\begin{eqnarray}
\frac{dx}{dt} & = & s(y-x) \\
\frac{dy}{dt} & = & rx-y-xz \\
\frac{dz}{dt} & = & xy-bz 
\end{eqnarray}

**2)** Sea $\vec{x}(t)=(x(t),y(t),z(t))$ y $p=(s,r,b)$. Usando el método de Runge-Kutta de orden 4 aquí proveído, resuelva la ODE para:

&nbsp; &nbsp; **i)** $\vec{x}(0)=(1,0.5,0.1)$, $p=(10,0.5,8/3)$ y $t\in [0,10]$.

&nbsp; &nbsp; **ii)** $\vec{x}(0)=(1,0.5,0.1)$ y $p=(10,10,8/3)$ y $t\in [0,20]$.

&nbsp; &nbsp; **iii)** $\vec{x}(0)=(1,0.5,0.1)$ y $p=(10,28,8/3)$ y $t\in [0,50]$

**3)** Para cada caso calculado en el inciso **2)**, grafique las curvas en función del tiempo $x(t)$, $y(t)$ y $z(t))$ en un mismo gráfico.

**4)** Para cada caso calculado en el inciso **2)**, grafique en paramétricamente en función del tiempo, y en 3 dimensiones, la trayectoria de la solución $\vec{x}(t)=(x(t),y(t),z(t))$.

**5)** Repita el caso **2.iii)** para $t\in [0,300]$ eliminando el transiente.

**6)** Identifique los máximos locales de la curva $z(t)$ obtenida en **5)**, y enumérelos como $z_1,z_2,...,z_i,...,z_n$.

**7)** Grafique $z_{i+1}$ vs $z_i$ para todo $i\in\{1,2,...,n-1\}$.

**8)** Con la intención de imitar un set de datos, agregue ruido Gaussiano de varianza $\sigma^2=1$ a la curva $z(t)$ vs $t$ generada en el inciso **2)**. Llamaremos a esta nueva curva $\tilde{z}(t)$ vs $t$. Grafíquela.

**9)** Utilice algún algoritmo de suavizado de curvas para suavizar $\tilde{z}(t)$ vs $t$.

**10)** Utilice la curva suavizada del inciso **9)** para identificar máximos locales correspondientes, $\tilde{z}_1$, $\tilde{z}_2$, ...,  $\tilde{z}_n$.

**11)** Grafique $\tilde{z}_{i+1}$ vs $\tilde{z}_i$ para todo $i\in\{1,2,...,n-1\}$.

**12)** Compare el gráfico del inciso **11)** con el del inciso **7)**, discuta y comente.

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
LATEX = True
import matplotlib.pyplot as plt
if LATEX:
  import matplotlib
  #from matplotlib import rc
  matplotlib.rc('text', usetex=True)
  matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
  if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng    
  else:
    print('Not running on CoLab')  
# %matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (7,5) # Dimensions of the figure
plt.rcParams['figure.dpi'] = 100 # Resolucion

## Integrador ODE

In [ ]:
def rk4(f,x,t,h,p):
    """
    Calcula un paso de integración del método de Runge Kutta orden 4.
    
    Argumentos de entrada:
    
        f : R^n -> R^n
        x = x(t) : R^n
        t = tiempo : R
        h = paso de tiempo : R
        p = parametros : R^q        
        
    Retorna aproximacion numérica de
    
        x(t+h) : R^n

    # Ejemplos:
    """    
    k1 = f(x,t,p)
    k2 = f(x+0.5*h*k1,t+0.5*h,p)
    k3 = f(x+0.5*h*k2,t+0.5*h,p)
    k4 = f(x+h*k3,t+h,p)
    
    return x+h*(k1+2.0*k2+2.0*k3+k4)/6.0

In [ ]:
def integrador_ode(m,f,x0,a,b,k,p):
    """
    Integra numéricamente la ODE
    
        dx/dt = f(x,t)
        
    sobre el intervalo t:[a,b] usando k pasos de integración y el método m, bajo condicion inicial x(a)=x0.
    No es necesario que a<b.
    
    Argumentos de entrada:
    
        m = metodo de integracion (ej. euler, rk2, etc.)
        f : R^n -> R^n
        x0 = condicion inicial : R
        a = tiempo inicial : R
        b = tiempo final : R
        k = num. pasos de integracion : N
        p = parametros : R^q
    
    Retorna:
    
        t : R^{k+1} , t_j = a+j*h para j=0,1,...,k
        x : R^{n,k+1} , x_ij = x_i(t_j) para i=0,1,...,n-1 y j=0,1,...,k
        
    donde a+k*dt = b.
    """  
    assert k>0
    n = len(x0)
    h = (b-a)/k
    x = np.zeros((n,k+1)) # Produce un array con forma y tipo especificada con los parametros, 
                          # lleno de ceros. la forma puede ser espcificada con un entero o tupla (n,k+1)    
    t = np.zeros(k+1)
    x[:,0] = x0           # actualiza la posicion inicial (columna de indice 0) de las variables con los valores 
                          # de las condiciones iniciales
    t[0] = a              # actualiza la posicion cero con el valor del tiempo inicial
    
    for j in range(k):    #Aca se produce la iteración en j 
        
        t[j+1] = t[j] + h   # iteracion tiempo 
        x[:,j+1] = m(f,x[:,j],t[j],h,p) # iteracion de x 
        
    return t,x

## Teoría

bla bla

## Rta. 1) Implementación del lado de derecho de la ODE

In [ ]:
def f(r,t,p):
    """
    r[0] = x(t)    : 
    r[1] = y(t)    : 
    r[2] = z(t)    : 
    t              : tiempo
    p[0] = s
    p[1] = r
    p[2] = b
    Retorna [dx/dt,dy/dt,dz/dt]
    """
    x = r[0]
    y = r[1]
    z = r[2]
    s = p[0]
    r = p[1]
    b = p[2]
    return np.array([
      s*(y-x),
      r*x-y-x*z,
      x*y-b*z
    ])

## Rta. 2) y 3)

In [ ]:
# a)
s = 10.0
r = 0.5
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 0.5
z0 = 0.1
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 10 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
plt.xlabel("$t$")
plt.ylabel("")
plt.plot(t,x,label='x',linestyle='-')
plt.plot(t,y,label='y',linestyle='-')
plt.plot(t,z,label='z',linestyle='-')
plt.legend()
plt.show()

In [ ]:
# b)
s = 10.0
r = 10.0
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 0.5
z0 = 0.1
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 20 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
plt.xlabel("$t$")
plt.ylabel("")
plt.plot(t,x,label='x',linestyle='-')
plt.plot(t,y,label='y',linestyle='-')
plt.plot(t,z,label='z',linestyle='-')
plt.legend()
plt.show()

In [ ]:
# c)
s = 10.0
r = 28.0
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 1.0
z0 = 1.0
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 50 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
plt.xlabel("$t$")
plt.ylabel("")
plt.plot(t,x,label='x',linestyle='-')
plt.plot(t,y,label='y',linestyle='-')
plt.plot(t,z,label='z',linestyle='-')
plt.legend()
plt.show()

## Rta. 4)

In [ ]:
# a)
s = 10.0
r = 0.5
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 1.0
z0 = 1.0
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 10 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x,y,z)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
plt.show()

In [ ]:
# b)
s = 10.0
r = 10.0
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 1.0
z0 = 1.0
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 20 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x,y,z)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
plt.show()

In [ ]:
# c)
s = 10.0
r = 28.5
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 1.0
z0 = 0.1
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 50 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
x,y,z = r
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x,y,z)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
plt.show()

## Rta. 5)

In [ ]:
# c)
s = 10.0
r = 28.0
b = 8.0/3.0
p = np.array([s,r,b])
x0 = 1.0
y0 = 1.0
z0 = 1.0
r0 = np.array([x0,y0,z0])
# Integramos
tini = 0 # h
tend = 300 # h
dt = 0.01 # h
nump = int((tend-tini)/dt) 
t,r = integrador_ode(rk4,f,r0,tini,tend,nump,p)
t_trans = 50
z = r[2][t_trans:]
t = t[t_trans:]

## Rta. 6)

In [ ]:
tmaxs = [t[i] for i in range(1,len(z)-1) if z[i]>z[i-1] and z[i]>z[i+1]]
zmaxs = [z[i] for i in range(1,len(z)-1) if z[i]>z[i-1] and z[i]>z[i+1]]

In [ ]:
plt.xlabel("$t$")
plt.ylabel("$z(t)$")
plt.plot(t,z,label='z',linestyle='-')
plt.scatter(tmaxs,zmaxs,label='z',linestyle='-',c='r',s=10)
plt.xlim(100,120)
#plt.legend()
plt.show()

## Rta. 7)

In [ ]:
plt.xlabel("$z_t$")
plt.ylabel("$z_{t+1}$")
plt.scatter(zmaxs[:-1],zmaxs[1:],label='z',linestyle='-')
plt.plot([0,50],[0,50],c='r')
plt.xlim(25,50)
plt.ylim(25,50)
#plt.legend()
plt.show()

## Rta. 8)

In [ ]:
# Simulamos los datos agregando ruido a la curva z(t) obtenida en el inciso 5).
tdata = t
zdata = z + 1.0*np.random.normal(size=len(z))

In [ ]:
plt.xlabel("$t$")
plt.ylabel("$\tilde{z}(t)$")
plt.plot(tdata,zdata,label='z',linestyle='-')
plt.xlim(100,120)
#plt.xlim(100,110)
#plt.xlim(105,106)
#plt.legend()
plt.show()

## Rta. 9)

In [ ]:
import scipy.signal as sgn

In [ ]:
zsmooth = sgn.savgol_filter(zdata,35,3)
tsmooth = t[:len(zsmooth)]

## Rta. 10)

In [ ]:
zdatamean = zdata.mean()
idatamaxs = [i for i in range(1,len(zsmooth)-1) if zsmooth[i]>zsmooth[i-1] and zsmooth[i]>zsmooth[i+1] and z[i]>zdatamean]
tdatamaxs = [tdata[i] for i in idatamaxs]
zdatamaxs = [zdata[i] for i in idatamaxs]
zsmoothmaxs = [zsmooth[i] for i in idatamaxs]

In [ ]:
plt.xlabel("$t$")
plt.ylabel("$z(t)$")
plt.plot(tsmooth,zsmooth,label='z',linestyle='-')
plt.scatter(tdatamaxs,zdatamaxs,label='z',linestyle='-',c='b',s=10)
plt.scatter(tdatamaxs,zsmoothmaxs,label='z',linestyle='-',c='r',s=10)
plt.xlim(100,120)
#plt.xlim(100,110)
#plt.xlim(105,106)
#plt.legend()
plt.show()

## Rta. 11)

In [ ]:
plt.title("C. Elegans: data")
plt.xlabel("$\tilde{z}_t$")
plt.ylabel("$\tilde{z}_{t+1}$")
plt.scatter(zdatamaxs[:-1],zdatamaxs[1:],label='z',linestyle='-')
plt.plot([0,50],[0,50],c='r')
plt.xlim(25,50)
plt.ylim(25,50)
#plt.legend()
plt.show()

In [ ]:
plt.title("C. Elegans: smoothed")
plt.xlabel("$\tilde{z}_t$")
plt.ylabel("$\tilde{z}_{t+1}$")
plt.scatter(zsmoothmaxs[:-1],zsmoothmaxs[1:],label='z',linestyle='-')
plt.plot([0,50],[0,50],c='r')
plt.xlim(25,50)
plt.ylim(25,50)
#plt.legend()
plt.show()